# 8장 영상 매칭과 추적
## 8.1 비슷한 그림 찾기
### 8.1.1 평균 해시 매칭

- 평균 해시: 어떤 영상이든 동일한 크기의 하나의 숫자로 변환되는데, 이때 숫자를 얻기 위해 평균값을 이용한다는 뜻
    - 평균을 얻기 전, 특정한 크기로 축소
    - 그 다음, 픽셀 전체의 평균 값을 구하고,
        - 픽셀 < 평균 → 픽셀 == 0
        - 픽셀 > 평균 → 픽셀 == 1 로 변환
    - 0 또는 1로만 구성된 각 픽셀 값을 1행 1열로 변환 (한 줄로 늘어선 0과 1은 한 개의 2진수 숫자로 볼 수 있다.)

In [1]:
# 일반 이미지를 평균 해시로 변환
import cv2

img = cv2.imread('snoopy.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [2]:
#16 by 16으로 크기 축소
gray = cv2.resize(gray, (16,16))

#영상의 평균 값 구하기
avg = gray.mean()

#평균값을 0과 1로 변환
bin = 1 * (gray>avg)
print(bin)

[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0]
 [0 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [0 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0]
 [1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0]
 [0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]]


In [3]:
#2진수 문자열을 16진수 문자열로 변환
dhash = []

for row in bin.tolist():
    s = ''.join([str(i) for i in row])
    dhash.append('%02x'%(int(s,2)))
    
print(dhash)

['40', '7d0', 'f70', '1ffc', '67f8', '7ffe', '3ff8', '73f3', '7fe2', 'f7e3', '27e4', '7efe', '1e38', '1df4', '7d0', '3c0']


In [4]:
dhash = ''.join(dhash)
print(dhash)

407d0f701ffc67f87ffe3ff873f37fe2f7e327e47efe1e381df47d03c0


In [5]:
cv2.imshow('snoopy', img)
cv2.waitKey(0)

27

- 다양한 물체가 있는 테스트셋으로 진행해보자!

In [6]:
import cv2
import numpy as np
import glob

In [7]:
img = cv2.imread('snoopy.jpg')
cv2.imshow('query', img)

#비교할 영상들이 있는 경로
search_dir = r'C:\Users\ghk96\Desktop\Motion\101_ObjectCategories'

#이미지를 16 by 16 평균 해시로 변환
def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.resize(gray, (16,16))
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

#해밍 거리 측정 함수
def hamming_distance(a,b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    
    #같은 자리의 값이 서로 다른 것들의 합
    distance = (a!=b).sum()
    return distance

#스누피 사진의 해시 구하기
query_hash = img2hash(img)

#이미지 데이터셋 디렉토리의 모든 영상 파일 경로
img_path = glob.glob(search_dir + '/**/*.jpg')

for path in img_path:
    img = cv2.imread(path) #데이터셋 영상을 하나 읽어서 표시
    cv2.imshow('searching..', img)
    cv2.waitKey(5)
    
    #데이터셋 영상 한 개의 해시
    a_hash = img2hash(img)
    
    #해밍거리 산출
    dst = hamming_distance(query_hash, a_hash)
    
    if dst/256 < 0.25: #해밍거리가 25% 이내만 출력
        print(path, dst/256)
        cv2.imshow(path, img)
        
cv2.destroyWindow('searching..')
cv2.waitKey(0)
cv2.destroyAllWindows()
    

C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\BACKGROUND_Google\image_0291.jpg 0.2421875
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\BACKGROUND_Google\image_0314.jpg 0.22265625
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\BACKGROUND_Google\image_0374.jpg 0.19921875
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\BACKGROUND_Google\image_0443.jpg 0.2265625
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\barrel\image_0005.jpg 0.18359375
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\bonsai\image_0116.jpg 0.2421875
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\brain\image_0001.jpg 0.2421875
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\brain\image_0007.jpg 0.21484375
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\brain\image_0014.jpg 0.23046875
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\brain\image_0016.jpg 0.20703125
C:\Users\ghk96\Desktop\Motion\101_ObjectCategories\brain\image_0017.jpg 0.22265625
C:\Users\ghk96\Desktop\Motion\101_ObjectC

- 실무에 사용할 만큼 좋은 결과를 가져오지 못하는 것을 알 수 있었다..
    - 스누피랑 연꽃... 스누피랑 사람 얼굴... 충격이었다...
    
    
    
    
- 사진의 회전, 크기, 방향 등에 영향을 주지 않으면서 정확도를 높이려면 어떻게 해야할까?
    - 특징을 잘 나타내는 여러 개의 지점을 찾아 그 특징을 잘 표현하고 서술할 수 있는 여러 개의 숫자들로 변환해야한다.
    - 이를 **키 포인트(Key Point)**와 **특징 디스크립터(Feature Descriptor)**라고 한다.
    
- 특징점의 개수와 서술 방식에 따라서 매칭하는 방법도 다양한데, 다음에서 알아볼 것이다!